In [1]:
from datasets import load_dataset
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

import time
import pandas as pd
import numpy as np

## Step 0: Read in the data

In [2]:
python_data = load_dataset("code_search_net", "python")
python_data

DatasetDict({
    train: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 412178
    })
    test: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 22176
    })
    validation: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 23107
    })
})

In [3]:
# Use the training data for Python code only
data_train = python_data['train']
data_train_snippet = data_train['whole_func_string']

## Step 1: Setting up ElasticSearch

In [4]:
es = Elasticsearch("http://localhost:9200")
es.info().body

{'name': '02982a8770ec',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'TeGH1jOWT2OmsX5S9hiisA',
 'version': {'number': '8.7.0',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '09520b59b6bc1057340b55750186466ea715e30e',
  'build_date': '2023-03-27T16:31:09.816451435Z',
  'build_snapshot': False,
  'lucene_version': '9.5.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [5]:
data_train[0].keys()

dict_keys(['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'])

In [6]:
# Creating the mappings (structure) for the python index
mappings = {
    "properties": {
        'repository_name': {"type": "text"}, 
        'func_path_in_repository': {"type": "text"}, 
        'func_name': {"type": "text"}, 
        'whole_func_string': {"type": "text"}, 
        'language': {"type": "text"}, 
        'func_code_string': {"type": "text"}, 
        'func_code_tokens': {"type": "text"}, 
        'func_documentation_string': {"type": "text"}, 
        'func_documentation_tokens': {"type": "text"}, 
        'split_name': {"type": "text"}, 
        'func_code_url': {"type": "text"}
    }
}

In [9]:
# Creating the index python with the mappings above
es.indices.create(index="python", mappings=mappings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'python'})

## Step 2: Adding Data into ElasticSearch

In [10]:
start = time.time()

# Creating a list of dictionaries with all the data to be added in to the ElasticSearch index
bulk_data = []
for i, row in enumerate(data_train):
    bulk_data.append(
        {
            "_index": "python",
            "_id": i,
            "_source": {
                "repository_name": row['repository_name'],
                'func_path_in_repository': row['func_path_in_repository'], 
                'func_name': row['func_name'], 
                'whole_func_string': row['whole_func_string'], 
                'language': row['language'], 
                'func_code_string': row['func_code_string'], 
                'func_code_tokens': row['func_code_tokens'], 
                'func_documentation_string': row['func_documentation_string'], 
                'func_documentation_tokens': row['func_documentation_tokens'], 
                'split_name': row['split_name'], 
                'func_code_url': row['func_code_url']
            }
        }
    )
    
end = time.time()
print("Time taken to read data into bulk_data:", end - start)

Time taken to read data into bulk_data: 344.7798068523407


In [11]:
start = time.time()

# Adding data into the index
bulk(es, bulk_data)

end = time.time()
print("time taken to add data into the index:", end - start)

time taken to add data into the index: 305.79853415489197


In [14]:
# verifying that all data has been read into the python index properly
es.indices.refresh(index="python")
es.cat.count(index="python", format="json")

ListApiResponse([{'epoch': '1699932894', 'timestamp': '03:34:54', 'count': '412178'}])

## Step 3: Implementing the Search Function

In [15]:
def es_search(query, k=10):
    """
    Searches the data using ElasticSearch to find the k most similar documents to the query.
    Returns a list of the k most similar functions, along with their GitHub URLs and their similarity scores to the query
    """
    es_query = {
        "query": {
            "query_string": {
                "query": query
            }
        },
        "size": k
    }
    
    response = es.search(index="python", body=es_query)
    
    results = []
    
    # for each result, add the function name, the GitHub URL of the function, and the similarity score to the results list
    for hit in response['hits']['hits']:
        row = hit['_source']
        results.append((row['func_name'], row['func_code_url'], hit['_score']))
        
    return results

## Step 4: Testing the Search Engine

In [16]:
query = "unique elements"
es_search(query, 5)

[('BaseProvider.random_choices',
  'https://github.com/joke2k/faker/blob/965824b61132e52d92d1a6ce470396dbbe01c96c/faker/providers/__init__.py#L207-L223',
  16.35838),
 ('BaseProvider.random_sample',
  'https://github.com/joke2k/faker/blob/965824b61132e52d92d1a6ce470396dbbe01c96c/faker/providers/__init__.py#L243-L248',
  16.35838),
 ('unique',
  'https://github.com/has2k1/plydata/blob/d8ca85ff70eee621e96f7c74034e90fec16e8b61/plydata/utils.py#L215-L256',
  15.654947),
 ('unique',
  'https://github.com/odlgroup/odl/blob/b8443f6aca90e191ba36c91d32253c5a36249a6c/odl/util/utility.py#L1573-L1611',
  15.207234),
 ('unique_everseen',
  'https://github.com/bsolomon1124/pyfinance/blob/c95925209a809b4e648e79cbeaf7711d8e5ff1a6/pyfinance/utils.py#L768-L776',
  15.006733)]

In [17]:
query = "create cookie"
es_search(query, 3)

[('WHTTPCookieJar.import_header_text',
  'https://github.com/a1ezzz/wasp-general/blob/1029839d33eb663f8dec76c1c46754d53c1de4a9/wasp_general/network/web/cookies.py#L316-L324',
  16.109276),
 ('IIIFAuth.access_token',
  'https://github.com/zimeon/iiif/blob/9d10018d01202fa2a76dfa61598dc6eca07b471f/iiif/auth.py#L257-L269',
  15.582415),
 ('make_cookie',
  'https://github.com/IdentityPython/pysaml2/blob/d3aa78eeb7d37c12688f783cb4db1c7263a14ad6/src/saml2/httputil.py#L320-L346',
  15.16988)]

## Step 5: Evaluating the Search Engine

In [18]:
def run_test_query_python_es(query_list, k=10):
    """
    Takes in a list of Python queries. Runs the search engine on those queries and returns
    the top k results for that query
    """
    test_results = []
    i = 1
    
    total_start = time.time()
    for query in query_list:
        # get top k results of query in our elastic search engine
        query_search = es_search(query, k)
        
        # for each result of the query, add a row to test_results with
        # the language (python), the query, and the GitHub Url to the result
        for result in query_search:
            query_results = ["python", query, result[1]]
            test_results.append(query_results)
        
        
    total_end = time.time()
    print("Time taken for all queries:", total_end - total_start)
    return test_results

In [19]:
# read in the annotated test dataset and get only the Python queries
test_queries = pd.read_csv('annotation_store.csv')
python_queries = test_queries[test_queries['Language'] == 'Python']

query_list = python_queries['Query'].unique().tolist()

In [26]:
# run the tests for the evaluation data
test_es_results = run_test_query_python_es(query_list, 300)

Time taken for all queries: 11.806987762451172


In [27]:
# store the test results as a Pandas DataFrame
test_es_results_df = pd.DataFrame(test_es_results)
test_es_results_df = test_es_results_df.rename(columns={0: "language", 1: "query", 2: "url"})
test_es_results_df

,language,query,url
0,python,sorting multiple arrays based on another array...,https://github.com/pyviz/holoviews/blob/ae0dd2...
1,python,sorting multiple arrays based on another array...,https://github.com/google/prettytensor/blob/75...
2,python,sorting multiple arrays based on another array...,https://github.com/rkday/nose2dep/blob/135a529...
3,python,sorting multiple arrays based on another array...,https://github.com/Zitrax/nose-dep/blob/fd29c9...
4,python,sorting multiple arrays based on another array...,https://github.com/lrq3000/pyFileFixity/blob/f...
...,...,...,...
29695,python,convert html to pdf,https://github.com/reingart/pyafipws/blob/ee87...
29696,python,convert html to pdf,https://github.com/doakey3/DashTable/blob/744c...
29697,python,convert html to pdf,https://github.com/chimera0/accel-brain-code/b...
29698,python,convert html to pdf,https://github.com/IndicoDataSolutions/IndicoI...


In [28]:
# Export the model predictions to a csv file
test_es_results_df.to_csv("es_model_predictions.csv")

**Current best model:**
- Using 200 top results from ElasticSearch engine
```
% of URLs in predictions that exist in the annotation dataset:
        python: 31.24%
% of URLs in predictions that exist in the annotation dataset (avg relevance > 0):
        python: 32.42%
NDCG:
        python: 0.355
NDCG (full ranking):
        python: 0.203
```